<a href="https://colab.research.google.com/github/InesAgudoMartin/APRENDIZAJE_AUTOMATICO_23_24/blob/main/LAB01_Preprocesamiento_y_visualizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LAB01 Preprocesamiento y visualización

María Rivas Ramos

Inés Agudo

Miguel Egido

Noah García Villanueva


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Usamos esto para que se nos muestren todos los atributos
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Carga de dataset en crudo, usando el método read_csv method.
# Más info: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

data_file_path = '/content/drive/MyDrive/MINERIA DE DATOS/PRACTICA 1/AviationData_practica1-1.csv'

full_data = pd.read_csv(data_file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MINERIA DE DATOS/PRACTICA 1/AviationData_practica1-1.csv'

## Exploración de los datos

### Exploración del dataset

In [ ]:
# El tipo de datos en el que lo hemos cargado es un DataFrame object.
# Más info: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

print(type(full_data))

In [ ]:
# Podemos visualizar el contenido

full_data

In [ ]:
# Resumen de información que hay en el DataFrame

full_data.info()


### Variables que nos interesan
Estas son las variables clave que utilizaremos en el análisis, ya que contienen información relevante sobre la que se pregunta

- **Event.Id**
- **Event.Date**
- **Country**
- **Injury.Severity**
- **Total.Fatal.Injuries**
- **Total.Serious.Injuries**
- **Total.Minor.Injuries**
- **Weather.Condition**
- **Broad.phase.of.flight**


### Variables que no nos interesan
Estas variables no son relevantes para el análisis, ya sea porque contienen información redundante, identificadores únicos sin utilidad analítica o detalles que no afectan a nuestras preguntas de investigación.

- **Accident.Number**
- **Investigation.Type**
- **Location**
- **Latitude**
- **Longitude**
- **Airport.Code**
- **Airport.Name**
- **Aircraft.damage**
- **Aircraft.Category**
- **Registration.Number**
- **Make**
- **Model**
- **Amateur.Built**
- **Number.of.Engines**
- **Engine.Type**
- **FAR.Description**
- **Schedule**
- **Purpose.of.flight**
- **Air.carrier**
- **Total.Uninjured**
- **Report.Status**
- **Publication.Date**


## Limpieza de los datos: duplicados

El único atributo que nos interesa que no esté duplicado es "Event.id"

In [ ]:
# Buscamos una de los atributos candidatos a tener duplicados. Por ejemplo 'Event.Id'
# Esto nos genera una serie de booleanos que indica si hay o no duplicado

duplicate_values = full_data['Event.Id'].duplicated()

print(duplicate_values.unique())

In [ ]:
# Podemos mostrar la primera ocurrencia de los duplicados

print(full_data[full_data.duplicated(subset='Event.Id')])

In [ ]:
# El método drop_duplicates borra los duplicados
# 1. Si keep = 'first' mantiene la primera ocurrencia

new_data_set = full_data.drop_duplicates(
    subset = ['Event.Id'],
    keep = 'first',
    inplace = False) # Esto significa que hace una copia del DataFrame

new_data_set.info()

print(new_data_set.loc[new_data_set['Event.Id']==63])

In [ ]:
# COMPROBAMOS QUE SE HAN ELIMINADO LOS DUPLICADOS CORRECTAMENTE

duplicate_values = new_data_set['Event.Id'].duplicated()

print(duplicate_values.unique())

## Ánalisis de los NA de cada atributo

In [ ]:
# Contar los valores nulos en cada columna
nans_count = new_data_set.isnull().sum()

# Mostrar en forma de tabla
nans_count.to_frame(name="Cantidad de NaNs")

Observamos que nuestra base de datos tiene una gran cantidad de NANs por lo que vamos a proceder a ignorar los atributos que no son necesarios y más a delante a completar los datos que las variables que sí nos interesan pero que debido a los datos faltantes puede dar error.

En el caso de los atributos que son necesarios para el análisis crearemos una categoría llamada "desconocido" y sustituiremos los valores desconocidos.

Procedemos a analizar y limpiar las siguientes variables:

- Event.Date
- Country
- Injury.Severity
- Total.Fatal.Injuries
- Total.Serious.Injuries
- Total.Minor.Injuries
- Weather.Condition
- Broad.phase.of.flight


### Event.Date

In [ ]:
print(new_data_set["Event.Date"].head())  # Ver las primeras fechas
print(new_data_set["Event.Date"].dtype)   # Ver el tipo de dato actual

In [ ]:
new_data_set.loc[:, "Event.Date"] = pd.to_datetime(new_data_set["Event.Date"], format="%Y-%m-%d", errors="coerce")

In [ ]:
# Comprobamos que no hay valores nulos
print(new_data_set["Event.Date"].isnull().sum())


In [ ]:
#  comprobamos que se ha guardado bien
print(new_data_set["Event.Date"].head())  # Ver las primeras fechas
print(new_data_set["Event.Date"].dtype)   # Ver el tipo de dato actual

### Injury.serevity

In [ ]:
injury_severity_counts = new_data_set["Injury.Severity"].value_counts(dropna=False)
display(injury_severity_counts.to_frame(name="Cantidad"))


In [ ]:
# Reemplazar los valores NaN en la columna "Injury.Severity" por "Unavailable"
new_data_set.loc[:, "Injury.Severity"] = new_data_set["Injury.Severity"].fillna("Unavailable")


# Verificar el cambio
injury_severity_counts = new_data_set["Injury.Severity"].value_counts()

# Mostrar los resultados actualizados
display(injury_severity_counts.to_frame(name="Cantidad"))


Para este atributo hemos observado que había NA pero también una variable llamada Unavailable, por lo que las hemos fusionado. Podemos observar como hemos eliminado la categoria de NA y la hemos juntado con Unavailable.

### Country

In [ ]:
# Contar la cantidad de ocurrencias de cada categoría en las columnas seleccionadas
country_counts = new_data_set["Country"].value_counts(dropna=False)

display(country_counts.to_frame(name="Cantidad"))


Vamos a normalizar los nombres poniendolos en minusculas y sin espacios para en el caso de que haya repeticiones no se diferencien por errores de trancripción.

In [ ]:
# Normalizar "Country": convertir a minúsculas y eliminar espacios extra
new_data_set.loc[:, "Country"] = new_data_set["Country"].str.strip().str.lower()


In [ ]:
# Contar la cantidad de ocurrencias de cada país y ordenarlos alfabéticamente
country_counts_sorted = new_data_set["Country"].value_counts(dropna=False).sort_index()

# Mostrar los resultados en orden alfabético
display(country_counts_sorted.to_frame(name="Cantidad"))


Vamos a unir los NA con los Unknown pues significan lo mismo.
No se juntan a missing porque aun que no se sepa a que país corresponde consideramos que es un dato importante pues representa aviones que se han perdido.

In [ ]:
# Reemplazar los valores NaN en la columna "Country" por "Unknown"
new_data_set.loc[:, "Country"] = new_data_set["Country"].fillna("unknown")

# Verificar el cambio y ordenarlos alfabéticamente
country_counts_sorted = new_data_set["Country"].value_counts(dropna=False).sort_index()

# Mostrar los resultados en orden alfabético
display(country_counts_sorted.to_frame(name="Cantidad"))

Podemos observar como ahora hay 225 Unknown incluyendo los anteriores NA.

Tras observar los paises, hemos observado algunos fallos que debemos corregir, los fallos son los siguientes:

- Hay un pais que se llama Un --> asumimos que es unknown

- "Turks And Caicos" y "Turks and Caicos Islands las juntamos pues todo el territorio son islas.

- "Korea, Republic Of", "South Korean" y "South Korea" los unimos en un solo país

- "Cote D'ivoire" y "Côte d'Ivoire los juntamos pues son el mismo con un acento de diferencia (^)

- Niger y Nigeria son el mismo país, los unimos

- block 651a es una dirección de Singapore, por lo que lo incluimos en ese país

In [ ]:
# Reemplazar "Un" con "unkown"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"un": "unknown"})

In [ ]:
# Reemplazar "Turks And Caicos" con "Turks and Caicos Islands"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"turks and caicos": "turks and caicos islands"})

In [ ]:
# Reemplazar "Cote D'ivoire" con "Côte d'Ivoire"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"cote d'ivoire": "côte d'ivoire"})

In [ ]:
# Reemplazar "Korea, Republic Of" con "South Korea"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"korea, republic of": "south korea"})

In [ ]:
# Reemplazar "South Korean" con "South Korea"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"south korean": "south korea"})

In [ ]:
# Reemplazar "Niger" con "Nigeria"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"niger": "nigeria"})

In [ ]:
# Reemplazar "block 651a" con "Singapore"
new_data_set.loc[:, "Country"] = new_data_set["Country"].replace({"block 651a": "singapore"})

In [ ]:
# Contar la cantidad de ocurrencias de cada país y ordenarlos alfabéticamente
country_counts_sorted = new_data_set["Country"].value_counts(dropna=False).sort_index()

# Mostrar los resultados en orden alfabético
display(country_counts_sorted.to_frame(name="Cantidad"))

### Weather.Condition


Vemos que hay UNK y Unk que son la misma pir lo que hay que cambiarlo y unirlos con NA

In [ ]:
weather_condition_counts = new_data_set["Weather.Condition"].value_counts(dropna=False)
display(weather_condition_counts.to_frame(name="Cantidad"))

In [ ]:
# Reemplazar "Unk" con "UNK"
new_data_set.loc[:, "Weather.Condition"] = new_data_set["Weather.Condition"].replace({"Unk": "UNK"})

# Reemplazar valores NaN con "UNK"
new_data_set.loc[:, "Weather.Condition"] = new_data_set["Weather.Condition"].fillna("UNK")

# Verificar el cambio
weather_condition_counts = new_data_set["Weather.Condition"].value_counts(dropna=False)

# Mostrar los resultados actualizados
display(weather_condition_counts.to_frame(name="Cantidad"))

### Broad.phase.of.flight


unir unk na y otros

In [ ]:
phase_condition_counts = new_data_set["Broad.phase.of.flight"].value_counts(dropna=False)
display(phase_condition_counts.to_frame(name="Cantidad"))

In [ ]:
# Reemplazar valores NaN con "Unknown"
new_data_set.loc[:, "Broad.phase.of.flight"] = new_data_set["Broad.phase.of.flight"].fillna("Unknown")


# Verificar el cambio
phase_condition_counts = new_data_set["Broad.phase.of.flight"].value_counts(dropna=False)

# Mostrar los resultados actualizados
display(phase_condition_counts.to_frame(name="Cantidad"))


In [ ]:
# Reemplazar "Unknown" con "Other"
new_data_set.loc[:, "Broad.phase.of.flight"] = new_data_set["Broad.phase.of.flight"].replace({"Unknown": "Other"})

In [ ]:
# Verificar el cambio
phase_condition_counts = new_data_set["Broad.phase.of.flight"].value_counts(dropna=False)

# Mostrar los resultados actualizados
display(phase_condition_counts.to_frame(name="Cantidad"))

### Total.Fatal.Injuries


In [ ]:
# Seleccionar las columnas relevantes
severity_injuries_table = new_data_set[["Injury.Severity", "Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries"]]

# Mostrar la tabla
display(severity_injuries_table.head())  # Muestra las primeras filas

In [ ]:
fatal_injuries_nans = new_data_set["Total.Fatal.Injuries"].isnull().sum()
print(f"NAn en 'Total.Fatal.Injuries': {fatal_injuries_nans}")


Como tenemos muchos nan vamos a comparar los valores nan con los valores de injury.severity para ver si se pudiesen completar

In [ ]:
# Filtrar filas donde "Total.Fatal.Injuries" es NaN
nan_fatal_injuries = new_data_set[new_data_set["Total.Fatal.Injuries"].isnull()]

# Contar los valores de "Injury.Severity" en esas filas
severity_for_nan_fatal = nan_fatal_injuries["Injury.Severity"].value_counts(dropna=False)

# Mostrar los resultados
display(severity_for_nan_fatal.to_frame(name="Cantidad"))


Vemos que ma mayoria de los casos son non-fatal o incidente por lo que vamos a asumir que no hubo victimas morales y sustituiremos los nan por 0.
Sin embargo en el caso de unavailable vamos a dejar los nan pues no disponemos de información.

In [ ]:
# Reemplazar NaN en "Total.Fatal.Injuries" por 0 solo si "Injury.Severity" es "Non-Fatal" o "Incident"
new_data_set.loc[new_data_set["Injury.Severity"].isin(["Non-Fatal", "Incident"]), "Total.Fatal.Injuries"] = 0

# Verificar los cambios
print(new_data_set["Total.Fatal.Injuries"].isnull().sum())  # Contar cuántos NaN quedan

### Total.Serious.Injuries


In [ ]:
Serious_injuries_nans = new_data_set["Total.Serious.Injuries"].isnull().sum()
print(f"NAn en 'Total.Serious.Injuries': {Serious_injuries_nans}")

Como tenemos muchos nan vamos a comparar los valores nan con los valores de injury.severity para ver si se pudiesen completar

In [ ]:
# Filtrar filas donde "Total.Serious.Injuries" es NaN
nan_serious_injuries = new_data_set[new_data_set["Total.Serious.Injuries"].isnull()]

# Contar los valores de "Injury.Severity" en esas filas
severity_for_nan_serious = nan_serious_injuries["Injury.Severity"].value_counts(dropna=False)

# Mostrar los resultados
display(severity_for_nan_serious.to_frame(name="Cantidad"))

### Total.Minor.Injuries

In [ ]:
minor_injuries_nans = new_data_set["Total.Minor.Injuries"].isnull().sum()
print(f"NAn en 'Total.Minor.Injuries': {minor_injuries_nans}")

In [ ]:
# Filtrar filas donde "Total.Serious.Injuries" es NaN
nan_minor_injuries = new_data_set[new_data_set["Total.Minor.Injuries"].isnull()]

# Contar los valores de "Injury.Severity" en esas filas
severity_for_nan_minor = nan_minor_injuries["Injury.Severity"].value_counts(dropna=False)

# Mostrar los resultados
display(severity_for_nan_minor.to_frame(name="Cantidad"))

tanto en seriour como en minor sabemos que hay tanto accidentes fatales como no fatales como unknown pero no se puede saber que valor corresponde a los datos vacios ni se puede poner 0 pues desajustaría los datos por lo que trabajaremos con estos nan.

##3.1. Análisis de Accidentes Aéreos en la Historia Reciente

###1- Cuestiones generales

¿Cómo ha evolucionado el número de accidentes y fatalidades?

¿Cuáles son los 5 países con más accidentes? Y ¿los 5 países con más víctimas mortales?

¿Podemos identificar si la aviación es más segura hoy en día en comparación con décadas pasadas?

Justifica las visualizaciones elegidas para soportar las conclusiones en tu informe.

###2- Causas de los accidentes

¿Las condiciones meteorológicas influyen en la gravedad de los accidentes? ¿Los accidentes fatales son más
frecuentes en condiciones adversas (IMC, vuelo instrumental) o en buen clima (VMC, vuelo visual)?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Contar los accidentes por tipo de clima y severidad
weather_severity_counts = new_data_set.groupby(['Weather.Condition', 'Injury.Severity']).size().unstack()

# Crear gráfico de barras
weather_severity_counts.plot(kind='bar', figsize=(8, 5), colormap='coolwarm')

# Personalización del gráfico
plt.title('Accidentes fatales según las condiciones meteorológicas')
plt.xlabel('Condición Meteorológica')
plt.ylabel('Número de accidentes')
plt.xticks(rotation=0)
plt.legend(title='Gravedad del accidente')
plt.show()

Genera gráficos que ayuden a interpretar cómo la meteorología afecta los accidentes

¿En qué fase del vuelo ocurren más accidentes morales? Y, por lo tanto, ¿Qué fase tiene más riesgo el
despegue, el aterrizaje o el crucero?

Genera un gráfico que muestre la distribución de accidentes fatales por fase del vuelo.

¿Los accidentes en tierra (Taxi, Standing) son realmente menos peligrosos que en el aire?

¿Las aeronaves no comerciales (Amateur Built) tienen una mayor proporción de accidentes en comparación
con las comerciales?

###3- Comparación de la evolución

¿Cómo ha evolucionado el número de accidentes en EEUU a lo largo de los años? ¿Y el número de víctimas mortales?

Compáralo con Brasil, México y Reino Unido ¿existe la misma tendencia en estos países en cuanto al número
de incidentes? ¿Y a las víctimas fatales?

##3.2. Análisis de la felicidad por reducción de la dimensionalidad

###1- Aplica el método de PCA y responde a las preguntas

¿Cuántas componentes principales debemos utilizar para tener más de un 80% de la variabilidad explicada?
Justifica tu respuesta.

¿Cuánta información perdemos si nos quedamos con 2 componentes? ¿Cuánta información representamos
con 3 componentes? Justifica tus respuestas.

###2- Selecciona 3 componentes principales y haz una representación gráfica

Obtén una representación de las tres primeras componentes. Explica lo que significan y cómo se relacionan
con los atributos originales

Por último, elige 2 países y explica lo que ocurre con ellos según sus componentes principales. Contrasta las
hipótesis con los datos originales